In [1]:
# Documentation: https://www.crummy.com/software/BeautifulSoup/bs4/doc/

from bs4 import BeautifulSoup
import requests

page = requests.get("https://www.starwars.com/news/star-wars-the-clone-wars-chronological-episodeorder/")

soup = BeautifulSoup(page.content)

# Print the first 950 chars of the page.
print(soup.prettify()[:950])



<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js lt-ie9 lt-ie8 lt-ie7" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js lt-ie9 lt-ie8" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js lt-ie9" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!-->
<html lang="en-US">
 <!--<![endif]-->
 <head>
  <meta charset="utf-8"/>
  <!-- Google Chrome Frame for IE -->
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <!-- mobile meta -->
  <meta content="True" name="HandheldFriendly"/>
  <meta content="320" name="MobileOptimized"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <!-- icons & favicons -->
  <meta content="Star Wars" name="apple-mobile-web-app-title"/>
  <link href="/apple-touch-icon.png" rel="apple-touch-icon"/>
  <link href="/apple-touch-icon.png" rel="icon" sizes="196x196"/>
  <link href="//a.dilcdn.com/sw/favicon-44660648232d.ico" rel="shortcut icon"/>
  <me


In [2]:

print(soup.title.string)


Star Wars: The Clone Wars Chronological Episode Order | StarWars.com


In [26]:

def begins_with_number(x):
    numbers = ["1","2","3","4","5","6","7","8","9","10"];
    return x[0] in numbers


headers = ["episode_code", "season","episode_number", "episode_name", "episode_detail_page_link"]

all_episodes = []

# for row in soup.find_all('td', class_="xl65")[0:3]:
for row in soup.find_all('td', class_="xl65"):
    if(row.string and begins_with_number(row.string)):
        ep_code = row.string
        season = ep_code[0]
        ep_number = int(ep_code[1:])
        
        a_link = row.parent.find("a")
        episode_name = a_link.text
        episode_detail_page_link = a_link['href']
                        
        print(ep_code,":", season, "-", ep_number)


        print(episode_name)
        print(episode_detail_page_link)

        episode = [ep_code,season, ep_number, episode_name, episode_detail_page_link]
        all_episodes.append(episode)
        print(" ")

        
        

216 : 2 - 16
Cat and Mouse
https://www.starwars.com/tv-shows/clone-wars/cat-and-mouse-episode-guide
 
116 : 1 - 16
Hidden Enemy
https://www.starwars.com/tv-shows/clone-wars/the-hidden-enemy-episode-guide
 
301 : 3 - 1
Clone Cadets
https://www.starwars.com/tv-shows/clone-wars/clone-cadets-episode-guide
 
303 : 3 - 3
Supply Lines
https://www.starwars.com/tv-shows/clone-wars/supply-lines-episode-guide
 
101 : 1 - 1
Ambush
https://www.starwars.com/tv-shows/clone-wars/ambush-episode-guide
 
102 : 1 - 2
Rising Malevolence
https://www.starwars.com/tv-shows/clone-wars/rising-malevolence-episode-guide
 
103 : 1 - 3
Shadow of Malevolence
https://www.starwars.com/tv-shows/clone-wars/shadow-of-malevolence-episode-guide
 
104 : 1 - 4
Destroy Malevolence
https://www.starwars.com/tv-shows/clone-wars/destroy-malevolence-episode-guide
 
105 : 1 - 5
Rookies
https://www.starwars.com/tv-shows/clone-wars/rookies-episode-guide
 
106 : 1 - 6
Downfall of a Droid
https://www.starwars.com/tv-shows/clone-wars/do

In [27]:
import pandas as pd

df = pd.DataFrame(all_episodes, columns=headers)
df.head()



,episode_code,season,episode_number,episode_name,episode_detail_page_link
0,216,2,16,Cat and Mouse,https://www.starwars.com/tv-shows/clone-wars/c...
1,116,1,16,Hidden Enemy,https://www.starwars.com/tv-shows/clone-wars/t...
2,301,3,1,Clone Cadets,https://www.starwars.com/tv-shows/clone-wars/c...
3,303,3,3,Supply Lines,https://www.starwars.com/tv-shows/clone-wars/s...
4,101,1,1,Ambush,https://www.starwars.com/tv-shows/clone-wars/a...


In [5]:
def clean(text):
    if(not text):
        return text

    cleaned = text.strip().replace('\n', ' ').replace('\r', '')
    return cleaned
    

def parse_detailed_info_page(url):
    
    #     request the page
    detail_page = requests.get(url)
    detail_soup = BeautifulSoup(detail_page.content)

    #description div has a quote and a description.
    description_div = detail_soup.find('div', class_="desc")
        
    start_quote_span = description_div.find('span', 'quote')
    if(start_quote_span):
        start_quote = start_quote_span.text
    else:
        start_quote = ""
        
    description = ""
    
#   first child is empty
#   2nd child is the quote
#   3rd child (index 2) is the description text. but it has some leading white-space.
    for index, child in enumerate(description_div.children):
        if(index==2):
            description = child.strip() 
            
    if((not start_quote) and (not description)):
        description = description_div.text.strip()
       
    #print((start_quote, str(description)))
    return (clean(start_quote), clean(description))


# Test the function works on a few examples
df["episode_detail_page_link"].sample(n=3).map(parse_detailed_info_page)


117    ("What is lost is often found.", A Jedi missio...
34     ("Attachment is not compassion.", Medical supp...
99     ("You must trust in others or success is impos...
Name: episode_detail_page_link, dtype: object

In [6]:
# Multi-thread it to apply it quickly across the dataframe and run requests in parallel.
import multiprocessing as mp
with mp.Pool(mp.cpu_count()) as pool:
    quotes_and_description = pool.map(parse_detailed_info_page, df["episode_detail_page_link"])

In [28]:
df2 = df
df2['quote'], df2['description'] = zip(*quotes_and_description)
df2.head()

,episode_code,season,episode_number,episode_name,episode_detail_page_link,quote,description
0,216,2,16,Cat and Mouse,https://www.starwars.com/tv-shows/clone-wars/c...,"""A wise leader knows when to follow""",A Separatist blockade surrounds Christophsis. ...
1,116,1,16,Hidden Enemy,https://www.starwars.com/tv-shows/clone-wars/t...,"""Truth enlightens the mind, but won't always b...",Anakin Skywalker and Obi-Wan Kenobi lead Repub...
2,301,3,1,Clone Cadets,https://www.starwars.com/tv-shows/clone-wars/c...,"""Brothers in arms are brothers for life.""",Five headstrong clones struggle to complete th...
3,303,3,3,Supply Lines,https://www.starwars.com/tv-shows/clone-wars/s...,"""Where there's a will, there's a way.""","Ryloth is under siege. Trapped on the surface,..."
4,101,1,1,Ambush,https://www.starwars.com/tv-shows/clone-wars/a...,"""Great leaders inspire greatness in others.""",Jedi Master Yoda and three clone troopers -- J...


In [29]:
display(df2)
df2.to_csv("clone_wars_episode_guide.csv")

,episode_code,season,episode_number,episode_name,episode_detail_page_link,quote,description
0,216,2,16,Cat and Mouse,https://www.starwars.com/tv-shows/clone-wars/c...,"""A wise leader knows when to follow""",A Separatist blockade surrounds Christophsis. ...
1,116,1,16,Hidden Enemy,https://www.starwars.com/tv-shows/clone-wars/t...,"""Truth enlightens the mind, but won't always b...",Anakin Skywalker and Obi-Wan Kenobi lead Repub...
2,301,3,1,Clone Cadets,https://www.starwars.com/tv-shows/clone-wars/c...,"""Brothers in arms are brothers for life.""",Five headstrong clones struggle to complete th...
3,303,3,3,Supply Lines,https://www.starwars.com/tv-shows/clone-wars/s...,"""Where there's a will, there's a way.""","Ryloth is under siege. Trapped on the surface,..."
4,101,1,1,Ambush,https://www.starwars.com/tv-shows/clone-wars/a...,"""Great leaders inspire greatness in others.""",Jedi Master Yoda and three clone troopers -- J...
5,102,1,2,Rising Malevolence,https://www.starwars.com/tv-shows/clone-wars/r...,"""Belief is not a matter of choice, but of conv...",An attack by a devastating new Separatist weap...
6,103,1,3,Shadow of Malevolence,https://www.starwars.com/tv-shows/clone-wars/s...,"""Easy is the path to wisdom for those not blin...",With the help of his Padawan Ahsoka and Jedi M...
7,104,1,4,Destroy Malevolence,https://www.starwars.com/tv-shows/clone-wars/d...,"""A plan is only as good as those who see it th...",Padmé Amidala and C-3PO are taken hostage by G...
8,105,1,5,Rookies,https://www.starwars.com/tv-shows/clone-wars/r...,"""The best confidence builder is experience.""","Alone on a distant outpost, clone officers Rex..."
9,106,1,6,Downfall of a Droid,https://www.starwars.com/tv-shows/clone-wars/d...,"""Trust in your friends, and they'll have reaso...",R2-D2 is lost during a fierce space battle -- ...


In [14]:
ep_code = "201"

In [15]:
season = ep_code[0]

In [22]:
print(season)
print(int(ep_code[1:]))

2
1
